<a href="https://colab.research.google.com/github/neennera/AI_pitch/blob/main/PyTorchForAudioML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Dowload Dataset

In [3]:
def download_mnist():
  train_dataset = datasets.MNIST(
      root="data", #ระบุที่เก็บ data ที่พึ่ง dowload มาที่ /data
      download = True, #ระบุให้โหลดมาหากยังไม่ได้โหลด
      train = True,
      transform=ToTensor()
  )
  test_dataset = datasets.MNIST(
      root="data",
      download = True,
      train =  False, # ให้โหลด test
      transform=ToTensor()
  )
  return train_dataset, test_dataset

In [14]:
class FeedForwardNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten() #แปลง array 2D ให้เป็น 1D
    self.dense_layer = nn.Sequential(  #การ pack layer เข้าด้วยกัน
        nn.Linear(28*28,256),  #784 -> 256
        nn.ReLU(),
        nn.Linear(256,10)
    )
    self.softmax = nn.Softmax(dim=1) #จำแนกการทาย class เป็น % 10 class [รวมกันได้ 1]
  
  def forward(self, x):
    x = self.flatten(x)
    x = self.dense_layer(x)
    x = self.softmax(x)
    return x

In [22]:
batch_size = 128
epochs = 10
lr = 0.001
device = 'cuda' if torch.cuda.is_available() else 'cpu'

if __name__ == "__main__" :
  train_dataset, test_dataset = download_mnist()
  train_dataloader=DataLoader(train_dataset,batch_size=batch_size)
  model = FeedForwardNet().to(device)

# Train model

In [23]:
loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=lr)

In [18]:
def train_one_epoch(model, data_loader, loss_fn, optimiser, device):
  for inputs, targets in data_loader :
    inputs, targets = inputs.to(device), targets.to(device)
  
  #calculate loss
  pred = model(inputs)
  loss = loss_fn(pred, targets)
  #acc = Acc_fn(pred, targets)

  #backpropagate + update weight
  optimiser.zero_grad() #ลบ gradiun จาก batch ก่อนๆ
  loss.backward()
  optimiser.step()

  print(f"Loss : {loss.item()}") #\t Acc:{acc}")

def train(model, data_loader, loss_fn, optimiser, device, epoch):
  for i in range(epoch):
    print(f"Epoch : {i+1}")
    train_one_epoch(model, data_loader, loss_fn, optimiser, device)
    print("---------------------------")
  print("Finished training")

In [25]:
train(model, train_dataloader, loss_fn, optimiser, device, epochs)
torch.save(model.state_dict(), "model.pth")
print("Trained feed forward net saved at feedforwardnet.pth")

Epoch : 1
Loss : 2.2604339122772217
---------------------------
Epoch : 2
Loss : 2.241356372833252
---------------------------
Epoch : 3
Loss : 2.218689203262329
---------------------------
Epoch : 4
Loss : 2.191959857940674
---------------------------
Epoch : 5
Loss : 2.1612331867218018
---------------------------
Epoch : 6
Loss : 2.1268694400787354
---------------------------
Epoch : 7
Loss : 2.089576005935669
---------------------------
Epoch : 8
Loss : 2.0501389503479004
---------------------------
Epoch : 9
Loss : 2.009643316268921
---------------------------
Epoch : 10
Loss : 1.969164252281189
---------------------------
Finished training
Trained feed forward net saved at feedforwardnet.pth
